# Setup Section

Unzip Dataset

In [ ]:
!unzip /content/fingers_dataset_modified.zip

Streaming output truncated to the last 5000 lines.
 extracting: fingers_dataset_modified/train/4/536e51bf-58ff-4003-9654-a9c5931afcf6_4L.png  
 extracting: fingers_dataset_modified/train/4/537517ab-6c0b-498d-ae81-3c8281e05d28_4R.png  
 extracting: fingers_dataset_modified/train/4/5375f545-8211-4e77-9c77-4cd7fa229b3b_4L.png  
 extracting: fingers_dataset_modified/train/4/53bda656-eba4-4746-8765-09a24be9ad33_4L.png  
 extracting: fingers_dataset_modified/train/4/53d23711-7124-4ad2-bece-d6e5650baba2_4L.png  
 extracting: fingers_dataset_modified/train/4/53efdc3c-e0c6-49e1-80ec-2208915bb7d9_4R.png  
 extracting: fingers_dataset_modified/train/4/54067598-dbc0-420f-bcf5-21ca6c153113_4L.png  
 extracting: fingers_dataset_modified/train/4/5407845b-7eff-4ec5-b9f5-b4fa12a5282f_4R.png  
 extracting: fingers_dataset_modified/train/4/54150b10-59c1-4242-b6fc-3ffb73dbcaa0_4L.png  
 extracting: fingers_dataset_modified/train/4/5422e7f1-5316-4a30-a7c1-21353d069988_4L.png  
 extracting: fingers_dataset_

Imports

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Sequential
import matplotlib.pyplot as plt
from keras_preprocessing import image
from tensorflow.python.keras import models
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.python.keras.layers import Dense, Dropout, Activation
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten

Constant Declarations

In [ ]:
IMG_SHAPE = (128,128)
KER_SHAPE = (5,5)
CLASS_COUNT = 6
DATA_DIR = "/content/fingers_dataset_modified"
BATCH_SIZE = 128

# Training Section

Image Preprocessing for Training and Validation Sets

In [ ]:
# The data_generator will scale the data and partition 20% of the images to be used for validation.
data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

# Collects the images from the training directory and applies preprocessing for training dataset
train_dataset = data_generator.flow_from_directory(
    DATA_DIR + "/train/", 
    target_size=IMG_SHAPE, 
    batch_size=BATCH_SIZE, 
    class_mode="categorical",
    color_mode="grayscale",
    subset="training",
    shuffle=True)

# Collects the images from the training directory and applies preprocessing for validation dataset
valid_dataset = data_generator.flow_from_directory(
    DATA_DIR + "/train/",
    target_size=IMG_SHAPE, 
    batch_size=BATCH_SIZE, 
    class_mode="categorical",
    color_mode="grayscale",
    subset="validation",
    shuffle=True)

Found 14400 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


Building the Model

In [ ]:
model = Sequential()

model.add( Conv2D(128,KER_SHAPE,input_shape=(128,128,1)))
model.add( Activation('relu'))

model.add( MaxPooling2D(2,2) )

model.add( Flatten() )

model.add( Dense(96) )
model.add( Activation('relu'))

model.add( Dropout(0.2) ) 

model.add( Dense(CLASS_COUNT) )
model.add( Activation('softmax') )

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Create callbacks

In [ ]:
# Used to stop the training
early_stopping = EarlyStopping(monitor='val_loss', mode='min', baseline=0.002)

In [ ]:
# **Use only for evaluating batch metrics with tensorboard**
#
# def batchOutput(batch, logs):
#     tf.summary.scalar('batch_loss', data=logs['loss'], step=batch)
#     tf.summary.scalar('batch_accuracy', data=logs['accuracy'], step=batch)
#     return batch

# batchTBCallback = LambdaCallback(on_batch_end=batchOutput)
# tensorboard = TensorBoard(log_dir=f"logs/FingersModel-{int(time.time())}",update_freq="batch")

Training the Model

In [ ]:
EPOCHS = 5
STEPS_PER_EPOCH = train_dataset.samples // BATCH_SIZE
V_STEPS = valid_dataset.samples // BATCH_SIZE

model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_dataset,
    validation_steps = V_STEPS,
    callbacks = [early_stopping], 
    #callbacks = [early_stopping,tensorboard,batchTBCallback], # **Use only for evaluating batch metrics with tensorboard**
    use_multiprocessing = True)

Epoch 1/5
112/112 [==============================] - 513s 5s/step - loss: 0.4758 - accuracy: 0.9074 - val_loss: 0.0079 - val_accuracy: 0.9989
Epoch 2/5
112/112 [==============================] - 526s 5s/step - loss: 0.0096 - accuracy: 0.9985 - val_loss: 0.0018 - val_accuracy: 0.9997


Save the Model

In [ ]:
model.save(filepath="FingersModel",overwrite=True)

INFO:tensorflow:Assets written to: FingersModel/assets


INFO:tensorflow:Assets written to: FingersModel/assets


Tensorboard Visualizaton

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs

# Testing Section

Testing the CNN

In [13]:
# Preparing and preprocessing the testing images
data_generator = ImageDataGenerator(rescale=1./255)
test_dataset = data_generator.flow_from_directory(
    DATA_DIR + "/test/",
    target_size=IMG_SHAPE, 
    batch_size=BATCH_SIZE, 
    class_mode="categorical",
    color_mode="grayscale",
    shuffle=False)

# Load the saved CNN model
#   This will only function if: 
#   - you have trained a model using the Training Section
#   OR
#   - you have uploaded the FingersModel file to the session storage
model = models.load_model("FingersModel")

# Prepare variables for prediction
filenames = test_dataset.filenames
sample_count = len(filenames)
T_STEPS = len(filenames) / BATCH_SIZE

# Get prediction data
prediction_data = model.predict(test_dataset,steps=T_STEPS)

# Prepare to display the predictions in an easy to ready fashion
prediction_indicies = np.argmax(prediction_data,axis=1)
labels = (test_dataset.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in prediction_indicies]

# Checks if predictions are correct based on the filename
#   The class of each file is embedded into the filename at the end, after the underscore.
#   "_1L" denotes and image of a "one" using the left hand. We have used this the verify
#   our predictions.
checks = [ filename[-6] == pred for filename,pred in zip(filenames,predictions) ]

# Create a dataframe to be printed that displays all prediction info
results = pd.DataFrame({"Filename":filenames, "Prediction":predictions, "Correct?":checks})

# Prints the final prediction results
print(results) # Only prints small part of dataframe
#print(results.to_string()) # Prints entire dataframe

# Prints ratio of correct predictions
true_count = np.count_nonzero(results['Correct?'])
print(f"The model predicted correctly {true_count} / {len(results['Correct?'])} images!")
print(f"Testing accuracy is {round(100*true_count/len(results['Correct?']),2)}%!")

Found 3600 images belonging to 6 classes.
                                           Filename Prediction  Correct?
0     0/00ab429e-7edd-4cf7-b6a6-25eec65d5cda_0L.png          0      True
1     0/00ea02b9-e3ed-43a4-ab8c-eb8a1172ca06_0L.png          0      True
2     0/00ebf839-9cfc-4b90-91ff-e57ec2e76ee3_0R.png          0      True
3     0/018a8903-ffb6-4028-a811-c9104be11943_0R.png          0      True
4     0/01ba21a1-40d7-4f6f-895a-1dd838cea3eb_0L.png          0      True
...                                             ...        ...       ...
3595  5/fcaaa0f9-635b-4f81-a477-b6351915c964_5L.png          5      True
3596  5/fd190327-7c2a-44a4-b96e-a55ce132c433_5R.png          5      True
3597  5/fda1b2c5-522d-42c8-8830-54c307e82432_5R.png          5      True
3598  5/fe1cc525-bac5-4c13-b6f3-7e1563a66402_5R.png          5      True
3599  5/feb4e57f-dc42-427a-a881-cb9af1d47dbf_5R.png          5      True

[3600 rows x 3 columns]
The model predicted correctly 3597 / 3600 images!
Testing